<a href="https://colab.research.google.com/github/urness/CS167Fall2025/blob/main/Day15_PCA_and_Perceptrons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS167: Day14
##Principal Component Analysis & Perceptrons

#### CS167: Machine Learning, Fall 2025


In [ ]:
# Mount your drive
from google.colab import drive
drive.mount('/content/drive')

## Principal Component Analysis Code

Documentation: [`sklearn.decomposition.PCA()`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)


In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

In [ ]:
# load the data
iris_df = pd.read_csv("/content/drive/MyDrive/CS167/datasets/irisData.csv")
predictors = ['sepal length', 'sepal width', 'petal length', 'petal width']
target = "species"


## First, let's establish a baseline -- How well does KNN do with all of the predictors?

In [ ]:
# split the data
train_data, test_data, train_sln, test_sln = \
  train_test_split(iris_df[predictors], iris_df[target], test_size = 0.2, random_state=29) # yes, I picked state 29 on purpose...
clf = KNeighborsClassifier()        # create classifier object
clf.fit(train_data,train_sln)       # fit the training data
predictions = clf.predict(test_data) # use the model to make predictions
print('Accuracy:',accuracy_score(test_sln,predictions)) # what is the accuracy?

## Now, let's use PCA to reduce the dimensions to just 2 principal components

In [ ]:
#whiten = True is important for uncorrelated
#attributes, and is False by default
extractor = PCA(n_components=2, whiten=True)
#When fitting with PCA, you do not use the target column - this is an unsupervised learning algorithm
extractor.fit(train_data)

print('this is the variance/importance of each component')
print(extractor.explained_variance_ratio_)

In [ ]:
print("Here's what the data looks like before being transformed:")
train_data[0:4]

In [ ]:
train_transformed = extractor.transform(train_data)

print("Here's what the training predictors look like after the transformation.")
train_transformed[0:4]

In [ ]:
#Now we transform the predictor columns in the test set as well.
#Notice that we're using the extractor that we trained using the training set.
#Do not re-fit it to the test data.
test_transformed = extractor.transform(test_data)

#Now we can use our transformed data with a classifier just like always:
clf = KNeighborsClassifier()
clf.fit(train_transformed,train_sln)
predictions = clf.predict(test_transformed)
print('Accuracy:',accuracy_score(test_sln,predictions))

In [ ]:
# visualize the training data in terms of the new principal components

import matplotlib.pyplot as plt
%matplotlib inline

#visualizing the new axes
#PCA gives it back as numpy array
tdf = pd.DataFrame(train_transformed)
#next line: probably not the best way
tdf['species'] = pd.Series(list(train_sln))


setosa_series = tdf[ tdf['species'] == 'Iris-setosa' ]
virginica_series = tdf[ tdf['species'] == 'Iris-virginica' ]
versicolor_series = tdf[ tdf['species'] == 'Iris-versicolor']

plt.plot(setosa_series[0],setosa_series[1],'ro',label='setosa')
plt.plot(virginica_series[0],virginica_series[1],'bs',label='virginica')
plt.plot(versicolor_series[0],versicolor_series[1],'g^',label='versicolor')
plt.legend(loc='upper center')
plt.show()

## PCA Code (all together)

In [ ]:
from sklearn.decomposition import PCA

# build your PCA extractor, decide how many output components (columns) you'd like
extractor = PCA(n_components=2, whiten=True)

#When fitting with PCA, you do not use the target column - this is an unsupervised learning algorithm
extractor.fit(train_data)

# transform your train_data and _test_data
train_transformed = extractor.transform(train_data)
test_transformed = extractor.transform(test_data)

#build and test your model
clf = KNeighborsClassifier()
clf.fit(train_transformed,train_sln)
predictions = clf.predict(test_transformed)
print('Accuracy:',accuracy_score(test_sln,predictions))

# 💬 Group Exercise:

- Download the the `boston_housing.csv` dataset from blackboard.
- See the code below to read in the data file and create the training and testing dataframes

In Blackboard, enter your answers for the the following R<sup>2</sup> metrics rounded to the nearest .01.

1. Establish a baseline. Without doing PCA, what is the R<sup>2</sup> metric for a **random forest** regressor, with `random_state = 4`?
2. Use PCA (as a pre-processing step) and a **random forest** regressor (with `random_state = 4`), what is the R<sup>2</sup> metric with using
   - 2 principal components?
   - 4 principal components?
   - 6 principal components?

In [ ]:
## Baseline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# load the data
housing_df = pd.read_csv("/content/drive/MyDrive/CS167/datasets/boston_housing.csv")
predictors = housing_df.columns.drop("MEDV")
target = "MEDV"

#split the data
train_data, test_data, train_sln, test_sln = \
       train_test_split(housing_df[predictors], housing_df[target], test_size = 0.2, random_state=41)